In [4]:
library(ggplot2)
library(dplyr)
library(reshape2)
library(yaml)

In [2]:
outputdir = file.path(getwd(), "..", read_yaml("../config.yaml", fileEncoding = "UTF-8")$outputDIR)

ERROR: Error in read_yaml("../config.yaml", fileEncoding = "UTF-8"): could not find function "read_yaml"


In [11]:
list.files(outputdir)

[1] "assembled"          "bbmap"              "cluster1"          
 [4] "cluster2"           "emptydir"           "fastqc"            
 [7] "firsttrim"          "jellyfish"          "megahit"           
[10] "merged"             "merged_all"         "qc"                
[13] "quast"              "renamed"            "salmon_indiv"      
[16] "transdecoder_indiv" "velvet"

In [17]:
indiv_salmon = file.path(outputdir, "salmon_indiv", "salmon_quant_assembly*", "aux_info", "meta_info.json")
system(paste0('grep --with-filename percent_mapped /', indiv_salmon, " > indiv_salmon.txt"))

In [45]:
salmon_res = read.csv("indiv_salmon.txt", sep = ":", header = FALSE, col.names = c("Path","Metric","Value"))
salmon_res$Path = as.character(salmon_res$Path)
salmon_res$Value = as.numeric(unlist(strsplit(trimws(as.character(salmon_res$Value)),",")))
for (curr_ind in c(1:length(salmon_res$Path))) {
    curr = suppressWarnings(unlist(strsplit(as.character(salmon_res$Path[curr_ind]), "/")))
    salmon_res$Path[curr_ind] = curr[length(curr)-2]
}
salmon_res

Path,Metric,Value
salmon_quant_assembly_SH254ad_megahit,percent_mapped,37.66273
salmon_quant_assembly_SH402_megahit,percent_mapped,60.28657
salmon_quant_assembly_SH402_trinity,percent_mapped,80.90828
salmon_quant_assembly_SH487_megahit,percent_mapped,54.14374
salmon_quant_assembly_SH487_trinity,percent_mapped,78.23583
salmon_quant_assembly_SH488_megahit,percent_mapped,59.43987
salmon_quant_assembly_SH488_trinity,percent_mapped,80.54100
salmon_quant_assembly_SH489_megahit,percent_mapped,53.00759
salmon_quant_assembly_SH489_trinity,percent_mapped,76.29318
salmon_quant_assembly_SH490_megahit,percent_mapped,57.87896


In [1]:
indiv_salmon = file.path(outputdir, "salmon", "salmon_quant_assembly*", "aux_info", "meta_info.json")
system(paste0('grep --with-filename percent_mapped /', indiv_salmon, " > indiv_salmon.txt"))

ERROR: Error in file.path(outputdir, "salmon_by_assembly_group", "salmon_quant_assembly*", : object 'outputdir' not found


In [46]:
cdhitdir = file.path(outputdir, 

ERROR: Error in parse(text = x, srcfile = src): <text>:1:77: unexpected symbol
1: cdhitdir = file.path(vortexfs1/omics/alexander/ncohen/BATS2019-clio-metaT/20Apr2020
                                                                                ^


In [47]:
ctdict = dict()
for curr in os.listdir(cdhitdir):
    resultscurr = pd.DataFrame(columns = ["Trinity","Velvet","Megahit","TV","TM","VM","TVM"])
    trinct = velvetct = megact = tvtog = tmtog = vmtog = tvmtog = 0
    if "clst" in curr:
        togglecol = pddict[curr][">Cluster 0"]
        ctr = 0
        print(len(togglecol))
        for t in togglecol:
            if ctr % 5000 == 0:
                print(str(trinct) + " , counter is " + str(ctr))
            ctr = ctr + 1
            if ">" not in str(t):
                if (trinct > 0) & (velvetct > 0):
                    tvtog = 1
                if (trinct > 0) & (megact > 0):
                    tmtog = 1
                if (velvetct > 0) & (megact > 0):
                    vmtog = 1
                if (velvetct > 0) & (megact > 0) & (trinct > 0):
                    tvmtog = 1
                resultscurr = resultscurr.append({'Trinity': trinct, 'Velvet': velvetct, 'Megahit': megact,\
                                   "TV": tvtog, "TM": tmtog, "VM": vmtog, "TVM": tvmtog}, ignore_index=True)
                trinct = velvetct = megact = tvtog = tmtog = vmtog = tvmtog = 0
                continue 
                
            if "trinity" in str(t):
                trinct = trinct + 1
            if "velvet" in str(t):
                velvetct = velvetct + 1
            if "megahit" in str(t):
                megact = megact + 1
        ctdict[curr] = resultscurr

ERROR: Error in parse(text = x, srcfile = src): <text>:2:5: unexpected symbol
1: ctdict = dict()
2: for curr
       ^


In [ ]:
for curr in ctdict.keys():
    print(curr)
    ctdict[curr]["VM"] = ((ctdict[curr]["Megahit"] > 0) & (ctdict[curr]["Velvet"] > 0)).astype(int)
    ctdict[curr]["TV"] = ((ctdict[curr]["Trinity"] > 0) & (ctdict[curr]["Velvet"] > 0)).astype(int)
    ctdict[curr]["TM"] = ((ctdict[curr]["Trinity"] > 0) & (ctdict[curr]["Megahit"] > 0)).astype(int)
    ctdict[curr]["TVM"] = ((ctdict[curr]["Trinity"] > 0) & (ctdict[curr]["Megahit"] > 0) & (ctdict[curr]["Velvet"] > 0)).astype(int)